## Extract ENS Data from BigQuery and Ethereum node

In [1]:
import pandas as pd
from ens import ENS
from web3 import Web3, HTTPProvider
from tqdm.notebook import tqdm
import ipfshttpclient
import content_hash

from config import ETH_URL, MN
from ens_data import get_ens_data
from src.utils_bigquery import get_df


EXTRACT_NAMES = False
EXTRACT_ENS_DATA = False
AGGREGATE_ENS_DATA = True

tqdm.pandas()

w3 = Web3(HTTPProvider(ETH_URL))
ns = ENS.fromWeb3(w3)
ipfs_client = ipfshttpclient.connect()

In [2]:
# 'ENS': '0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85'

### Get ENS Names from Blockchain ETL datasets (BigQuery)

In [3]:
query_1 = f'''
    SELECT DISTINCT name
    FROM (
        SELECT name
        FROM `blockchain-etl.ethereum_ens.ETHRegistrarController_event_NameRegistered`

        UNION ALL

        SELECT name
        FROM `blockchain-etl.ethereum_ens.ETHRegistrarController2_event_NameRegistered`

        UNION ALL

        SELECT name
        FROM `blockchain-etl.ethereum_ens.ETHRegistrarController3_event_NameRegistered`)
'''
if EXTRACT_NAMES:
    ens_names = get_df(query=query_1)
    ens_names.to_csv('data/ens_names.csv')
else:
    ens_names = pd.read_csv('data/ens_names.csv')

In [4]:
ens_names_list = ens_names.name.to_list()
ens_names_list[:10]

['ethercryptocurrency',
 'ufcfighters',
 'coinapp',
 '2rickychan',
 'lootvault',
 'arthurs',
 'feelsgoodman',
 'doodzuki',
 'fattigerworkshop',
 'bioart']

### ENS Data Extraction from Ethereum node
Use `ens_data.py` for getting ENS Data for names from Ethereum node.

### Aggregate ENS Data

In [5]:
if AGGREGATE_ENS_DATA:
    ens_data_df = None
    for i in tqdm(range(252)):
        try:
            with open(f'data/temp/ens_data_{i}.csv') as my_csv:
                ens_data_item_df = pd.read_csv(my_csv, index_col=0)
            if ens_data_df is None:
                ens_data_df = ens_data_item_df
            else:
                ens_data_df = ens_data_df.append(ens_data_item_df)
        except FileNotFoundError:
            print(f'Error {i} not found')
    ens_data_df.name = ens_data_df.name.map(lambda x: x.lower())
    ens_data_df = ens_data_df.drop_duplicates()
    ens_data_df = ens_data_df.sort_values('name').reset_index()
    ens_data_df = ens_data_df[['name', 'owner', 'resolver_address', 'associated_address', 'contenthash']]
    ens_data_df.to_csv('data/ens_data_row.csv')
else:
    ens_data_df = pd.read_csv('data/ens_data_row.csv', index_col=0)
ens_data_df.head(20)

  0%|          | 0/252 [00:00<?, ?it/s]

,name,owner,resolver_address,associated_address,contenthash
0,0.eth,0x03B7bA1730170e6120529902479A026f7CDB60C6,NaN,NaN,NaN
1,8.eth,0x00000000c9c32755D496d3F13DcDf06A436C9C7a,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x00000000c9c32755D496d3F13DcDf06A436C9C7a,NaN
2,6.eth,0x00000000c9c32755D496d3F13DcDf06A436C9C7a,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x00000000c9c32755D496d3F13DcDf06A436C9C7a,NaN
3,8.eth,0x00000000c9c32755D496d3F13DcDf06A436C9C7a,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x00000000c9c32755D496d3F13DcDf06A436C9C7a,NaN
4,0.eth,0x94567Fa65d284b51CE7832B3CD2682a2277dc964,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x94567Fa65d284b51CE7832B3CD2682a2277dc964,NaN
5,8.eth,0x7c6d5cf17D05fDD70fB051366a474F7b422d86a6,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x7c6d5cf17D05fDD70fB051366a474F7b422d86a6,NaN
6,.eth,0x30a0063FfBb27C11FFD22Dd5321E8B07b259083d,NaN,NaN,NaN
7,0.eth,0xf8d06F8338abA317afc9b55dF03D41Bf71f927Dd,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0xf8d06F8338abA317afc9b55dF03D41Bf71f927Dd,NaN
8,6969.eth,0x00000000c9c32755D496d3F13DcDf06A436C9C7a,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x00000000c9c32755D496d3F13DcDf06A436C9C7a,NaN
9,8.eth,0x4a083475Fd0BfB1eBB05d03D29794bc64696F3AF,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x4a083475Fd0BfB1eBB05d03D29794bc64696F3AF,NaN


### Get items with content hashes

In [6]:
def decode_contenthash(contenthash: str) -> str:
    try:
        content_cid = content_hash.decode(contenthash) if contenthash[:2] == 'e3' else ''
        content_cid = content_cid if len(content_cid) == 46 else ''
        return content_cid
    except ValueError:
        return ''


if AGGREGATE_ENS_DATA:
    ens_data_with_contenthash_df = ens_data_df[
        (~ens_data_df.contenthash.isna()) & (~ens_data_df.contenthash.isin(["b''"]))].copy().sort_index()
    ens_data_with_contenthash_df.loc[:, 'content_cid'] = ens_data_with_contenthash_df.contenthash.map(
        decode_contenthash)
    ens_data_with_contenthash_df.to_csv('data/ens_data_with_content_hash.csv')

    ens_data_with_cid_df = ens_data_with_contenthash_df[
        ens_data_with_contenthash_df.content_cid != ''].copy().sort_index()
    ens_data_with_cid_df.loc[:, 'name_cid'] = ens_data_with_cid_df.name.progress_map(
        lambda x: ipfs_client.add_str(x[:-4]))
    ens_data_with_cid_df.to_csv('data/ens_data_with_content_cid.csv')

    print(f'Count with content hash: {len(ens_data_with_contenthash_df):>,}')
    print(f'Count with cid hash: {len(ens_data_with_cid_df):>,}')
else:
    ens_data_with_contenthash_df = pd.read_csv('data/ens_data_with_content_hash.csv', index_col=0)
    ens_data_with_cid_df = pd.read_csv('data/ens_data_with_content_cid.csv', index_col=0)

  0%|          | 0/5607 [00:00<?, ?it/s]

Count with content hash: 11,340
Count with cid hash: 5,607


In [7]:
ens_data_with_cid_df.head(10)

,name,owner,resolver_address,associated_address,contenthash,content_cid,name_cid
66331,0-1337.eth,0x8F82De72659D5a1670d3Db7C440A8E0a6E925E02,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,NaN,e30101701220d6f0e0d157170f52052e3f81bdafafb2e3...,QmcogzR9ouQJDgsiaTWWze4ndMLkn6gewRqKVWeKXVozh7,QmUwjXxYcgk1j2Eh7jiVtgSGn3hmLHWoqNDLTonts6hJV8
71898,0000x.eth,0x77C9e49073d9830723195f3788570c2c36c97688,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x77C9e49073d9830723195f3788570c2c36c97688,e301017012203c7939754cbfabd4a7932a740b2af3ced2...,QmSQiPAPaT5E9y41VgHhc5kFRhRFSvZ95d5sBzH6VJwdPo,QmVzMe3mUGLB68Lz5wpQcfjab8akjQj7SdoE1ioAzfiw6E
75750,0008a.eth,0x88441DCD4Bc985965e69ecf5A454B04DC84eD876,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x88441DCD4Bc985965e69ecf5A454B04DC84eD876,e30101701220cdaf3eb20495dacca5225a9a0838207af7...,QmcBZGuZCQsd5zuumVRuLhXnhkWjWJiXtmcE9XQVsAXV8N,Qmawaesea9bHkCMmZupyMn5pbVgAJjKVM3MS2rUqZ1GhKf
78535,001x0.eth,0x338571a641d8c43F9e5A306300c5d89E0CB2CFAF,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x338571a641d8c43F9e5A306300c5d89E0CB2CFAF,e301017012209668df055d7566e1631a6db9d66015204a...,QmYTncViYfeLbyHS6EECfUoRW3kjqrbiUmKbZLZEXUmA6R,QmXk8AcZzo88TBa1TiJT3uLzcKnTY8DTyVa7euFmMh9qYw
79530,002511.eth,0x4E99f6Fdb8B5a3A292aa2194c970c2A612469Fe2,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x4E99f6Fdb8B5a3A292aa2194c970c2A612469Fe2,e301017012205952f152144216dfc3267c1f44130f1235...,QmUMLME8Lkipg1cqw8m3pwpWHhRN34ioFEbjbhw46oqkN2,QmNbqwb5T2oWTUjWMRWxUBAGYUvy3v6Ta7SHaz83Yo9TT6
79618,00258.eth,0x9e6D2A36fa88E5123fD09d22F00e4E9ca8669999,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x9e6D2A36fa88E5123fD09d22F00e4E9ca8669999,e301017012200429cb961fb23fd71947c477c881f63671...,QmNcuJbp2YLK6BmhdTniVsjjfxcgP16dGBKZeUHQQ6vHqB,QmUhnpZg3Wvz3eSTrAj66xVa3xrKb62CF63V5zWwu6Z3pa
89735,0089.eth,0xFebfecE019b5E2e42C3839dCbEA87EEDac9ac365,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0xFebfecE019b5E2e42C3839dCbEA87EEDac9ac365,e301017012200676503f43d887c1e1d20c23d988d4d941...,QmNmsnw5FpncDeUvJ3ZRFY3YtvEYWoCHWvgjEwjV41gUjM,QmSTGTAnKY5n3Xz4DEY1oWEmrVPuAXTp469L4bGVWyiX12
96266,0123a.eth,0x9aE6B470D238a4ab42cAc5fEB27CAA5A19F89510,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x9aE6B470D238a4ab42cAc5fEB27CAA5A19F89510,e30101701220c801aa770d8313b86fe43cb08472166a71...,QmboPirQwrzimcSfGbeRHxySy4sH19ZmoJywVSf7bH7n8T,QmaUedgdJdVrzAEGud28YFNAv6cuwnbdxBgG4WktfZNpSZ
97207,014°.eth,0x70635102CF3B7D32FE0a1221b412Cb4b172A2b2f,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x70635102CF3B7D32FE0a1221b412Cb4b172A2b2f,e3010170122086f1db4d21efaa7220ad3ec4460ac2b97b...,QmXRREtZRWsppmHwJqFYW9RfazaWh5mob5iRBgX1GLtcF9,QmWRhM2YPtfZbLNaW4Fk2JLuQBBFWNXDzWyzrYazYieE4r
99044,01x01.eth,0x8F82De72659D5a1670d3Db7C440A8E0a6E925E02,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,0x8F82De72659D5a1670d3Db7C440A8E0a6E925E02,e30101701220d6f0e0d157170f52052e3f81bdafafb2e3...,QmcogzR9ouQJDgsiaTWWze4ndMLkn6gewRqKVWeKXVozh7,QmSF6D6bPeR4FF8MiJgYYY5prufPh3B17g2bmpxZkFzWqy


In [8]:
for column in ens_data_df.columns:
    print(
        f'''{column}: {len(ens_data_df[(~ens_data_df[column].isna()) & (~ens_data_df[column].isin(["b''", '0x0000000000000000000000000000000000000000']))]):>,} ({len(ens_data_df[(~ens_data_df[column].isna()) & (~ens_data_df[column].isin(["b''", '0x0000000000000000000000000000000000000000']))]) / len(ens_data_df) * 100:.2f}%)''')
print(
    f'''\nassociated_address!=owner: {len(ens_data_df[(ens_data_df.associated_address != ens_data_df.owner) & (~ens_data_df.associated_address.isna())]):>,}''')

name: 2,512,258 (100.00%)
owner: 2,497,034 (99.39%)
resolver_address: 1,887,536 (75.13%)
associated_address: 1,865,670 (74.26%)
contenthash: 11,340 (0.45%)

associated_address!=owner: 46,499


### ENS Namespace Contracts
- name -> owner
- owner -> name
- name -> associated_address
- associated_address -> name
- name -> cid
- cid -> name
#### All Data

In [9]:
name_owner_cnt = len(ens_data_df[~ens_data_df.owner.isna()][['name', 'owner']])
name_associated_address_cnt = len(ens_data_df[~ens_data_df.associated_address.isna()])
name_contenthash_cnt = len(
    ens_data_with_contenthash_df[(~ens_data_with_contenthash_df.contenthash.isna())])
name_cid_cnt = len(
    ens_data_with_contenthash_df[ens_data_with_contenthash_df.content_cid != ''])
print(f"name -> owner:              {name_owner_cnt:>7,}  ({name_owner_cnt / len(ens_data_df):.2%})")
print(
    f"name -> associated_address: {name_associated_address_cnt:>7,}  ({name_associated_address_cnt / len(ens_data_df):.2%})")
print(f"name -> contenthash:        {name_contenthash_cnt:>7,}  ({name_contenthash_cnt / len(ens_data_df):.2%})")
print(f"name -> cid:                {name_cid_cnt:>7,}  ({name_cid_cnt / len(ens_data_df):.2%})")
print(
    f"Total cyberlinks:           {(name_owner_cnt + name_associated_address_cnt + name_contenthash_cnt + name_cid_cnt) * 2 :>7,}")

name -> owner:              2,511,205  (99.96%)
name -> associated_address: 1,865,670  (74.26%)
name -> contenthash:         11,340  (0.45%)
name -> cid:                  5,607  (0.22%)
Total cyberlinks:           8,787,644


#### Data with CIDs only

In [10]:
name_owner_cnt = len(ens_data_with_cid_df[~ens_data_with_cid_df.owner.isna()][['name', 'owner']])
name_associated_address_cnt = len(ens_data_with_cid_df[~ens_data_with_cid_df.associated_address.isna()])
name_contenthash_cnt = len(
    ens_data_with_cid_df)
name_cid_cnt = len(
    ens_data_with_cid_df)
print(f"name -> owner:              {name_owner_cnt:>7,}")
print(f"owner -> name:              {name_owner_cnt:>7,}\n")
print(f"name -> associated_address: {name_associated_address_cnt:>7,}")
print(f"associated_address -> name: {name_associated_address_cnt:>7,}\n")
print(f"name -> cid:                {name_cid_cnt:>7,}")
print(f"cid -> name:                {name_cid_cnt:>7,}\n")
print(f"Total cyberlinks:           {(name_owner_cnt + name_associated_address_cnt + name_cid_cnt) * 2 :>7,}")

name -> owner:                5,607
owner -> name:                5,607

name -> associated_address:   5,269
associated_address -> name:   5,269

name -> cid:                  5,607
cid -> name:                  5,607

Total cyberlinks:            32,966
